In [1]:
import pandas as pd
import numpy as np

# Recyclers CHM/KYIV

In [2]:
data = pd.read_csv("dec/data/fulls/Declarations_third_splitted_table.csv", sep=';', dtype={'dest_edrpou': object, 'value': object, 'value_prediction': object, 'value_destination': object})
data.fillna('', inplace=True)
data.head(4)

/home/roman/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,No,trash_name,code,alter_name,class,storage,value,value_prediction,destination,value_destination,operation,operation_desc,decl_num,dest_edrpou,dest_name,dest_address,data_fact_address,dest_date
0,1,"Масла та мастила моторні, трансмісійні нехлоро...",6000.2.8.09,Відпрацьовані мастила,ІІ,0,0.012,0.012,38563155 ТОВ «Науково- дослідний інститут «УКР...,0.012,R9,,D1221855500_00001_260716,38563155,ТОВ «Науково- дослідний інститут «УКРЕКОПРОЕКТ»,"69002, м. Запоріжжя, вул. Червоногвардійська, ...","69000, м. Запоріжжя, вул. Теплична, 27. ,",08.01.2014
1,2,"Матеріали обтиральні зіпсовані, відпрацьовані ...",7730.3.1.06,Промащене ганчір’я,ІІІ,0,0.100,0.211,38563155 ТОВ «Науково- дослідний інститут «УКР...,0.100,R5,,D1221855500_00001_260716,38563155,ТОВ «Науково- дослідний інститут «УКРЕКОПРОЕКТ»,"69002, м. Запоріжжя, вул. Червоногвардійська, ...","69000, м. Запоріжжя, вул. Теплична, 27. ,",08.01.2014
2,3,"Відходи комунальні (міські) змішані, у т. ч. с...",7720.3.1.01,Побутове сміття,ІV,0,18.000,84.18,38641065 КП «Чисте місто» Жовтоводської місько...,18.000,D1,,D1221855500_00001_260716,38641065,"КП «Чисте місто» Жовтоводської міської ради""","52200,м. Жовті Води, вул. Петровського,56",,
3,4,Підметини та пух хутра штучного,1830.2.1.06,Підметини з цехів,ІV,0,35.200,40.5,38563155 ТОВ «Науково- дослідний інститут «УКР...,35.200,R5,,D1221855500_00001_260716,38563155,ТОВ «Науково- дослідний інститут «УКРЕКОПРОЕКТ»,"69002, м. Запоріжжя, вул. Червоногвардійська, ...","69000, м. Запоріжжя, вул. Теплична, 27.",


In [3]:
data_dropped = data.drop_duplicates(subset=['dest_edrpou'], keep='last') # drop same edrpous except last to find out current address
# filter by address
edrs = data_dropped[((data_dropped['dest_address'].str.contains("київ|хмельницький|хмельницька", case=False)) & (~data_dropped['dest_address'].str.contains("вул. київська|вулиця київська|київський район|київський р-н", case=False))) | ((data_dropped['data_fact_address'].str.contains("київ|хмельницький|хмельницька", case=False)) & (~data_dropped['data_fact_address'].str.contains("вул. київська|вулиця київська|київський район|київський р-н", case=False)))]['dest_edrpou']

In [4]:
# data clean of value_destination
data['value_destination'] = data['value_destination'].replace('-', '0').replace('', '0').replace({'тонн':''}, regex=True).replace('0.120.', '0.12')

In [6]:
# only those in our range
data_mini = data[data['dest_edrpou'].isin(list(edrs))]

In [7]:
# dictionar creation
dicti = dict((el, {'overall': 0, 'class': dict((el, 0) for el in range(4)), 'R': dict((el, 0) for el in range(14)), 'D': dict((el, 0) for el in range(16))}) for el in data_mini['dest_edrpou'].unique())
roman_to_int = {'І': 1, 'ІІ': 2, 'ІІІ': 3, 'ІV': 4}
for index, row in data_mini.iterrows():
    dicti[row['dest_edrpou']]['class'][roman_to_int[row['class']] - 1] += float(row['value_destination'])
    dicti[row['dest_edrpou']]['overall'] += float(row['value_destination'])
    dicti[row['dest_edrpou']][row['operation'][0]][int(row['operation'][1]) - 1] += float(row['value_destination'])

In [8]:
# ranking by overall sum of waste tones
ranking = sorted(dicti.items(), key=lambda kv: kv[1]['overall'], reverse=True)

In [9]:
# creating new dataframe and save it
df = pd.DataFrame(columns=['EDRPOU', 'Name', 'Address', 'Actual Adress', 'Wastes taken(in tonnes)', 'Wastes of I class', 'Wastes of II class', 'Wastes of III class', 'Wastes of IV class', 'R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'R9', 'R10', 'R11', 'R12', 'R13', 'R14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D16'])
for i, j in ranking:
    row_dict = {'EDRPOU': i, 'Name': data[data['dest_edrpou']==i].iloc[-1]['dest_name'], 'Address': data[data['dest_edrpou']==i].iloc[-1]['dest_address'], 'Actual Adress': data[data['dest_edrpou']==i].iloc[-1]['data_fact_address'], 'Wastes taken(in tonnes)': j['overall'], 'Wastes of I class': j['class'][0], 'Wastes of II class': j['class'][1], 'Wastes of III class': j['class'][2], 'Wastes of IV class': j['class'][3]}
    r = dict(zip(['R' + str(i) for i in range(1, 15)], [j['R'][i] for i in range(14)]))
    d = dict(zip(['D' + str(i) for i in range(1, 17)], [j['D'][i] for i in range(16)]))
    row_dict.update(r)
    row_dict.update(d)
    df = df.append(row_dict, ignore_index=True)

In [10]:
# df[df['Wastes taken(in tonnes)'] != 0].to_excel("./dec/data/fulls/biggest_waste_recyclers2.xls",index=False, encoding="utf-8")
# df

# part 2 Generators

In [2]:
data1 = pd.read_csv("dec/data/fulls/Declarations_second_table.csv", sep=';')
data0 = pd.read_csv("dec/data/fulls/Declarations_first_table.csv", sep=';')

In [12]:
data1['amounts'] = data1['amount_I'] + data1['amount_II'] + data1['amount_III'] + data1['amount_IV']

In [14]:
data_locations = pd.DataFrame({"Address": list(data0[data0['0'] == "Юридична адреса суб’єкта господарювання:"]['1']), "EDRPOU": list(data0[data0['0'] == "Код згідно з ЄДРПОУ:"]['1'])})
data_locations.drop_duplicates("EDRPOU", keep='last', inplace=True)
edrs2 = data_locations[((data_locations['Address'].str.contains("київ|хмельницький|хмельницька", case=False)) & (~data_locations['Address'].str.contains("вул. київська|вулиця київська|київський район|київський р-н", case=False)))]['EDRPOU']

In [15]:
data_with_edrpou_and_decl_nums = data0[data0['0'] == 'Код згідно з ЄДРПОУ:'][data0[data0['0'] == 'Код згідно з ЄДРПОУ:']['1'].isin(edrs2.values.reshape(edrs2.size, ))][["1", "decl_num"]]

In [16]:
decs_needed = data_with_edrpou_and_decl_nums['decl_num']

In [17]:
data_mini2 = data1[data1['decl_num'].isin(list(decs_needed))]

In [18]:
# 0 - by index, 1 - sum of all classes, 2-5 - classes
dicti2 = dict((el,[0] * 6) for el in edrs2.values.reshape(edrs2.size, ))
for index, row in data_mini2.iterrows():
    dicti2[data_with_edrpou_and_decl_nums[data_with_edrpou_and_decl_nums['decl_num'] == row['decl_num']]['1'].values[0]][0] += float(row['index_gen_wastes'])
    dicti2[data_with_edrpou_and_decl_nums[data_with_edrpou_and_decl_nums['decl_num'] == row['decl_num']]['1'].values[0]][1] += float(row['amounts'])
    dicti2[data_with_edrpou_and_decl_nums[data_with_edrpou_and_decl_nums['decl_num'] == row['decl_num']]['1'].values[0]][2] += float(row['amount_I'])
    dicti2[data_with_edrpou_and_decl_nums[data_with_edrpou_and_decl_nums['decl_num'] == row['decl_num']]['1'].values[0]][3] += float(row['amount_II'])
    dicti2[data_with_edrpou_and_decl_nums[data_with_edrpou_and_decl_nums['decl_num'] == row['decl_num']]['1'].values[0]][4] += float(row['amount_III'])
    dicti2[data_with_edrpou_and_decl_nums[data_with_edrpou_and_decl_nums['decl_num'] == row['decl_num']]['1'].values[0]][5] += float(row['amount_IV'])

In [19]:
ranking2 = sorted(dicti2.items(), key=lambda kv: kv[1][0], reverse=True)

In [20]:
df2 = pd.DataFrame(columns=['EDRPOU', 'Name', 'Address', 'Contacts', 'KOATUU', 'Wastes produced in Показник загального утворення відходів (Пзув)', 'Wastes produced in tonnes', 'amount_I', 'amount_II', 'amount_III', 'amount_IV'])
for i, j in ranking2:
    df2 = df2.append({'EDRPOU': i, 'Name': data0.iloc[data0[data0['0']=='Код згідно з ЄДРПОУ:'][data0['1'] == i].iloc[-1].name - 1]['1'], "Address": data0.iloc[data0[data0['0']=='Код згідно з ЄДРПОУ:'][data0['1'] == i].iloc[-1].name + 2]['1'], 'Contacts': data0.iloc[data0[data0['0']=='Код згідно з ЄДРПОУ:'][data0['1'] == i].iloc[-1].name + 3]['1'], 'KOATUU': data0.iloc[data0[data0['0']=='Код згідно з ЄДРПОУ:'][data0['1'] == i].iloc[-1].name + 1]['1'], 'Wastes produced in Показник загального утворення відходів (Пзув)': j[0], 'Wastes produced in tonnes': j[1], 'amount_I': j[2], 'amount_II': j[3], 'amount_III': j[4], 'amount_IV': j[5]}, ignore_index=True)
#     df2 = df2.append({'EDRPOU': i, 'Name': data0[data0['decl_num'] == data0[data0['0']=='Код згідно з ЄДРПОУ:'][data0['1'] == i].iloc[-1]['decl_num']][data0['0'] == "Назва суб'єкта господарювання:"]['1'].iloc[0], 'Address': data0[data0['decl_num'] == data0[data0['0']=='Код згідно з ЄДРПОУ:'][data0['1'] == i].iloc[-1]['decl_num']][data0['0'] == "Юридична адреса суб’єкта господарювання:"]['1'].iloc[0], 'Contacts': data0[data0['decl_num'] == data0[data0['0']=='Код згідно з ЄДРПОУ:'][data0['1'] == i].iloc[-1]['decl_num']][data0['0'] == "Електронна пошта /Телефон:"]['1'].iloc[0], 'KOATUU': data0[data0['decl_num'] == data0[data0['0']=='Код згідно з ЄДРПОУ:'][data0['1'] == i].iloc[-1]['decl_num']][data0['0'] == "Код згідно з КОАТУУ:"]['1'].iloc[0], 'Wastes produced in Показник загального утворення відходів (Пзув)': j[0], 'Wastes produced in tonnes': j[1]}, ignore_index=True)

/home/roman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
# writer = pd.ExcelWriter('dec/data/fulls/biggest_waste_recyclers_and_generators.xls', engine='xlsxwriter')

# df.to_excel(writer, sheet_name='Recyclers')
# df2.to_excel(writer, sheet_name='Generators')

# writer.save()
df2

,EDRPOU,Name,Address,Contacts,KOATUU,Wastes produced in Показник загального утворення відходів (Пзув),Wastes produced in tonnes,amount_I,amount_II,amount_III,amount_IV
0,40075815,"Виробничий підрозділ ""Ужгородська дистанція ел...","м. Київ, вул. Тверська, 5 (м. Чоп, вул. Москов...",falkovskaya68@mail.ru/0312693480,2111000000,263597.8627,49380.6610,26.2662,144.5058,220.4913,48989.3977
1,30019801,АТ «УКРТРАНСГАЗ» філія УМГ «КИЇВТРАНСГАЗ» Бояр...,"01021, м. Київ, Узвіз Кловський, 9/1",ryadnovskaya-tn@utg.ua/(044)406-32-64,3222410300,137315.2739,8451.4426,11.8661,121.1070,185.9756,8132.4939
2,00135390,Публічне Акціонерне Товариство «УКРНАФТА»,"04053 м. Київ, провулок Несторівський, 3/5",Office@ukrnafta.com/(0432)52-51-53,0510100000,124406.4836,18486.8508,13.9892,25.2114,477.6966,17969.9536
3,40081195,"ВСП ""Рівненська дистанція колії"" АТ ""Українськ...","м.Київ, вул.Тверська,5",ushurik@gmail.com/42 23 80,5610100000,100529.6498,8415.5402,11.9089,63.4765,18.5050,8321.6498
4,23744453,"ПІДПРИЄМСТВО З ІНОЗЕМНИМИ ІНВЕСТИЦІЯМИ ""МакДон...","02140, м.Київ, вул. Гришка, буд. 7",ua-00034.store@ua.mcd.com/(0512)50-10-27,4810137200,94632.1784,47697.9452,6.5418,27.6000,9.3750,47654.4284
5,40081200,"ВП служби колії ""Знам'янська дистанція колії"" ...","03680 м. Київ вул. Тверська, 5",pch_10@ukr.net/05233-6-24-71,3510600000,82403.9210,4361.1256,6.4887,88.7373,27.0586,4238.8410
6,40982829,"Товариство з обмеженою відповідальністю ""Фудком""","01103, м. Київ, Залізничне шосе,57",Rokhmaniiko_N@kishenya.ua/067-413-24-04,8036100000,80509.9640,45876.8920,6.9280,0.0000,0.0000,45869.9640
7,36265925,"Дочірнє підприємство ""УКРАВТОГАЗ"" Національної...","03134, м.Київ, Святошинський р-н, вул.Григоров...",maznich@ukr.net/0503121869,5110137300,70836.1510,3810.8686,3.3246,52.0590,498.5340,3256.9510
8,30603572,Підприємство з іноземними інвестиціями «АМІК У...,"04071, м. Київ, Подільський р-н, вул. Верхній ...",esavchenko@amicenergy.com.ua/0504120216,2110100000,66416.3050,7977.3983,4.8142,0.0000,701.4841,7271.1000
9,32049199,"ТОВ ""Метро Кеш енд Кері Україна"" (філія у м.Од...","02140, м.Київ, пр. П.Григоренка, 43 (65033, м....",andrii.hrys@metro.ua/050-355-29-78,5110137300,61243.6030,25224.9959,6.2519,7.2750,23.1660,25188.3030


# All Recyclers

In [22]:
data = pd.read_csv("dec/data/fulls/Declarations_third_splitted_table.csv", sep=';', dtype={'dest_edrpou': object, 'value': object, 'value_prediction': object, 'value_destination': object})
data.fillna('', inplace=True)
data['value_destination'] = data['value_destination'].replace('-', '0').replace('о', '0').replace('', '0').replace({'тонн':''}, regex=True).replace('0.120.', '0.12')
data.head(5)

,No,trash_name,code,alter_name,class,storage,value,value_prediction,destination,value_destination,operation,operation_desc,decl_num,dest_edrpou,dest_name,dest_address,data_fact_address,dest_date
0,1,"Масла та мастила моторні, трансмісійні нехлоро...",6000.2.8.09,Відпрацьовані мастила,ІІ,0,0.012,0.012,38563155 ТОВ «Науково- дослідний інститут «УКР...,0.012,R9,,D1221855500_00001_260716,38563155,ТОВ «Науково- дослідний інститут «УКРЕКОПРОЕКТ»,"69002, м. Запоріжжя, вул. Червоногвардійська, ...","69000, м. Запоріжжя, вул. Теплична, 27. ,",08.01.2014
1,2,"Матеріали обтиральні зіпсовані, відпрацьовані ...",7730.3.1.06,Промащене ганчір’я,ІІІ,0,0.100,0.211,38563155 ТОВ «Науково- дослідний інститут «УКР...,0.100,R5,,D1221855500_00001_260716,38563155,ТОВ «Науково- дослідний інститут «УКРЕКОПРОЕКТ»,"69002, м. Запоріжжя, вул. Червоногвардійська, ...","69000, м. Запоріжжя, вул. Теплична, 27. ,",08.01.2014
2,3,"Відходи комунальні (міські) змішані, у т. ч. с...",7720.3.1.01,Побутове сміття,ІV,0,18.000,84.18,38641065 КП «Чисте місто» Жовтоводської місько...,18.000,D1,,D1221855500_00001_260716,38641065,"КП «Чисте місто» Жовтоводської міської ради""","52200,м. Жовті Води, вул. Петровського,56",,
3,4,Підметини та пух хутра штучного,1830.2.1.06,Підметини з цехів,ІV,0,35.200,40.5,38563155 ТОВ «Науково- дослідний інститут «УКР...,35.200,R5,,D1221855500_00001_260716,38563155,ТОВ «Науково- дослідний інститут «УКРЕКОПРОЕКТ»,"69002, м. Запоріжжя, вул. Червоногвардійська, ...","69000, м. Запоріжжя, вул. Теплична, 27.",
4,5,Плівка чи оболонка на основі полімерів зіпсова...,1753.1.1.03,Відходи поліетилену,ІV,0,1.912,2.000,38563155 ТОВ «Науково- дослідний інститут «УКР...,1.912,R5,,D1221855500_00001_260716,38563155,ТОВ «Науково- дослідний інститут «УКРЕКОПРОЕКТ»,"69002, м. Запоріжжя, вул. Червоногвардійська, ...","69000, м. Запоріжжя, вул. Теплична, 27.",


In [23]:
dicti = dict((el, {'overall': 0, 'class': dict((el, 0) for el in range(4)), 'R': dict((el, 0) for el in range(14)), 'D': dict((el, 0) for el in range(16))}) for el in data['dest_edrpou'].unique())
roman_to_int = {'І': 1, 'ІІ': 2, 'ІІІ': 3, 'ІV': 4}
for index, row in data.iterrows():
    dicti[row['dest_edrpou']]['class'][roman_to_int[row['class']] - 1] += float(row['value_destination'])
    dicti[row['dest_edrpou']]['overall'] += float(row['value_destination'])
    dicti[row['dest_edrpou']][row['operation'][0]][int(row['operation'][1]) - 1] += float(row['value_destination'])

In [24]:
ranking = sorted(dicti.items(), key=lambda kv: kv[1]['overall'], reverse=True)

In [25]:
df3 = pd.DataFrame(columns=['EDRPOU', 'Name', 'Address', 'Actual Adress', 'Wastes taken(in tonnes)', 'Wastes of I class', 'Wastes of II class', 'Wastes of III class', 'Wastes of IV class', 'R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'R9', 'R10', 'R11', 'R12', 'R13', 'R14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D16'])
for i, j in ranking:
    row_dict = {'EDRPOU': i, 'Name': data[data['dest_edrpou']==i].iloc[-1]['dest_name'], 'Address': data[data['dest_edrpou']==i].iloc[-1]['dest_address'], 'Actual Adress': data[data['dest_edrpou']==i].iloc[-1]['data_fact_address'], 'Wastes taken(in tonnes)': j['overall'], 'Wastes of I class': j['class'][0], 'Wastes of II class': j['class'][1], 'Wastes of III class': j['class'][2], 'Wastes of IV class': j['class'][3]}
    r = dict(zip(['R' + str(i) for i in range(1, 15)], [j['R'][i] for i in range(14)]))
    d = dict(zip(['D' + str(i) for i in range(1, 17)], [j['D'][i] for i in range(16)]))
    row_dict.update(r)
    row_dict.update(d)
    df3 = df3.append(row_dict, ignore_index=True)

# All Generators

In [26]:
data1 = pd.read_csv("dec/data/fulls/Declarations_second_table.csv", sep=';')
data0 = pd.read_csv("dec/data/fulls/Declarations_first_table.csv", sep=';')

In [27]:
data1['amounts'] = data1['amount_I'] + data1['amount_II'] + data1['amount_III'] + data1['amount_IV']

In [30]:
data_with_edrpou_and_decl_nums = data0[data0['0'] == 'Код згідно з ЄДРПОУ:'][["1", "decl_num"]]

In [31]:
decs_needed = data_with_edrpou_and_decl_nums['decl_num']

In [32]:
dicti2 = dict((el,[0] * 6) for el in data0[data0['0'] == 'Код згідно з ЄДРПОУ:']['1'].unique())
for index, row in data1.iterrows():
    dicti2[data_with_edrpou_and_decl_nums[data_with_edrpou_and_decl_nums['decl_num'] == row['decl_num']]['1'].values[0]][0] += float(row['index_gen_wastes'])
    dicti2[data_with_edrpou_and_decl_nums[data_with_edrpou_and_decl_nums['decl_num'] == row['decl_num']]['1'].values[0]][1] += float(row['amounts'])
    dicti2[data_with_edrpou_and_decl_nums[data_with_edrpou_and_decl_nums['decl_num'] == row['decl_num']]['1'].values[0]][2] += float(row['amount_I'])
    dicti2[data_with_edrpou_and_decl_nums[data_with_edrpou_and_decl_nums['decl_num'] == row['decl_num']]['1'].values[0]][3] += float(row['amount_II'])
    dicti2[data_with_edrpou_and_decl_nums[data_with_edrpou_and_decl_nums['decl_num'] == row['decl_num']]['1'].values[0]][4] += float(row['amount_III'])
    dicti2[data_with_edrpou_and_decl_nums[data_with_edrpou_and_decl_nums['decl_num'] == row['decl_num']]['1'].values[0]][5] += float(row['amount_IV'])

In [33]:
ranking2 = sorted(dicti2.items(), key=lambda kv: kv[1][0], reverse=True)

In [34]:
df4 = pd.DataFrame(columns=['EDRPOU', 'Name', 'Address', 'Contacts', 'KOATUU', 'Wastes produced in Показник загального утворення відходів (Пзув)', 'Wastes produced in tonnes', 'Відходи I класу', 'Відходи II класу', 'Відходи III класу', 'Відходи IV класу'])
for i, j in ranking2:
    df4 = df4.append({'EDRPOU': i, 'Name': data0.iloc[data0[data0['0']=='Код згідно з ЄДРПОУ:'][data0['1'] == i].iloc[-1].name - 1]['1'], "Address": data0.iloc[data0[data0['0']=='Код згідно з ЄДРПОУ:'][data0['1'] == i].iloc[-1].name + 2]['1'], 'Contacts': data0.iloc[data0[data0['0']=='Код згідно з ЄДРПОУ:'][data0['1'] == i].iloc[-1].name + 3]['1'], 'KOATUU': data0.iloc[data0[data0['0']=='Код згідно з ЄДРПОУ:'][data0['1'] == i].iloc[-1].name + 1]['1'], 'Wastes produced in Показник загального утворення відходів (Пзув)': j[0], 'Wastes produced in tonnes': j[1], 'Відходи I класу': j[2], 'Відходи II класу': j[3], 'Відходи III класу': j[4], 'Відходи IV класу': j[5]}, ignore_index=True)

/home/roman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [35]:
writer = pd.ExcelWriter('dec/data/fulls/biggest_waste_recyclers_and_generators_ALL.xls', engine='xlsxwriter')

df.to_excel(writer, sheet_name='Recyclers')
df2.to_excel(writer, sheet_name='Generators')
df3.to_excel(writer, sheet_name='RecyclersALL')
df4.to_excel(writer, sheet_name='GeneratorsALL')

writer.save()